In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_train = pd.read_csv("./train.csv")
df_test = pd.read_csv("./test.csv")
df = [df_train, df_test]

In [3]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


survival:   Survival
PassengerId: Unique Id of a passenger.
pclass: Ticket class    
sex:    Sex 
Age:    Age in years    
sibsp:  # of siblings / spouses aboard the Titanic  
parch:  # of parents / children aboard the Titanic  
ticket: Ticket number   
fare:   Passenger fare  
cabin:  Cabin number    
embarked:   Port of Embarkation

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [ ]:
df_test.info()

In [ ]:
# colomne index pour les futurs merge
for tab in df:
    tab.reset_index(inplace=True)

In [ ]:
# On remplace les NaN par la médiane des ages
for tab in df:
    tab["Age"].fillna(tab["Age"].median(), inplace=True)

In [ ]:
# Trop de NaN pour Cabin, on le drop
for tab in df:
    tab.drop(["Cabin"], axis=1, inplace=True)

In [ ]:
df_train["Embarked"].value_counts()

In [ ]:
# S est la classe la plus représentée
for tab in df:
    tab["Embarked"] = tab["Embarked"].fillna("S")

In [ ]:
#One hot encoder pour Embarked
for i in range (0, len(df)):
    df[i] = df[i].merge(pd.get_dummies(df[i]["Embarked"], 
                         drop_first=True).reset_index(), on="index")
    df[i].drop("Embarked", axis=1, inplace=True)

In [ ]:
# On remplace les données manquantes de Fare par la moyenne par class (à finir)
for tab in df:
    tab['Fare'] = tab['Fare'].fillna(tab["Fare"].mean())

In [ ]:
# Le ticket semble à priori ne rien apporter : on le drop. De même pour PassagerID
for tab in df:
    tab.drop(["Ticket", "PassengerId"], axis=1, inplace=True)

In [ ]:
# nombre de personne de la famille: Sibsp + Parch
for tab in df:
    tab["family_number"] = tab["Parch"] + tab["SibSp"]
    tab.drop(["Parch", "SibSp"], axis=1, inplace=True)

In [ ]:
for tab in df:
    tab["Sex"] = pd.get_dummies(tab["Sex"], drop_first=True)
# 1 pour homme, 0 pour femme

In [ ]:
#Pour l'instant je n'utilise pas la colomne "Name"
for tab in df:
    tab.drop("Name", axis=1, inplace=True)

In [ ]:
# On enlève la colomne "index" car tous les merge ont été effectués
for tab in df:
    tab.drop("index", axis=1, inplace=True)

# Classifier

In [ ]:
#from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def loss(y_pred, y_true):
    return np.sum(y_pred == y_true) / len(y_pred)

In [ ]:
X = df[0].drop("Survived", axis=1).values
y = df[0]["Survived"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_sub = df[1].values

### Regression Logistique

In [ ]:
# avec CV
clf = LogisticRegression()
scores = cross_val_score(clf, X, y, cv=5)
np.mean(scores)

### Decision Tree

In [ ]:
clf = DecisionTreeClassifier()
scores = cross_val_score(clf, X, y, cv=5, n_jobs=-1)
np.mean(scores)

### Random Forest

In [ ]:
clf = RandomForestClassifier(n_jobs=-1)
scores = cross_val_score(clf, X, y, cv=5, n_jobs=-1)
np.mean(scores)

### KNN

In [ ]:
clf = KNeighborsClassifier()
scores = cross_val_score(clf, X, y, cv=5, n_jobs=-1)
np.mean(scores)

### On prédit que les femmes vivent et que les hommes meurent

In [ ]:
df[1].head()

In [ ]:
y_pred = (1 - df[1]["Sex"]).values

In [ ]:
y_pred = []
for i in range(len(df[1])):

        y_pred.append(1)
    else
    

# Enregistrement des données


In [ ]:
clf = RandomForestClassifier(n_jobs=-1)
clf.fit(X, y)
y_pred = clf.predict(X_sub)

In [ ]:
PassengerId = df_test["PassengerId"]
submission = pd.DataFrame({
        "PassengerId": PassengerId,
        "Survived": y_pred
    })
submission.to_csv('gender_submission.csv', index=False)

In [ ]:
PassengerId